# Building a Spam Filter with Naive Bayes


## Introduction

In this project, we're going to study the practical side of the Naive Bayes algorithm by building a spam filter for SMS messages. To classify messages as spam or non-spam, the computer basically:

- Learns how humans classify messages.
- Uses that human knowledge to estimate probabilities for new messages — probabilities for spam and non-spam.
- Classifies a new message based on these probability values — if the probability for spam is greater, then it classifies the message as spam. Otherwise, it classifies it as non-spam (if the two probability values are equal, then we may need a human to classify the message).


Our first task is to _teach_ the computer how to classify messages. To do that, we'll use the multinomial Naive Bayes algorithm along with a [dataset](https://dq-content.s3.amazonaws.com/433/SMSSpamCollection) of 5,572 SMS messages that are already classified by humans.


Let's start by reading in the dataset. 

In [1]:
import pandas as pd
import re

In [2]:
spam_sms = pd.read_csv(
        'SMSSpamCollection',
        sep='\t',
        header = None,
        names=['Label', 'SMS']
        )
spam_sms.shape

(5572, 2)

In [3]:
spam_sms.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
spam_sms.tail()

,Label,SMS
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...
5571,ham,Rofl. Its true to its name


In [5]:
spam_sms['Label'].value_counts(normalize=True) * 100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

We can see above that about 87% of the content is labeled as _ham_, meaning __non-spam__, while the other 13% is _spam_.

## Dataset split

Let's move on to create a test for our software. It is better to do this now, in order to avoid biasing while creating the filter itself.

To test the spam filter, we're first going to split our dataset into two categories:

- A training set, which we'll use to "train" the computer how to classify messages. This will be 80% of the dataset, 4458 messages, in order to have an extensive training.
- A test set, which we'll use to test how good the spam filter is with classifying new messages. This will be the remaining 20%, meaning 1114 messages.

As this dataset has been human-classified, we can test a portion of it with the algorithm, and then compare the results (_machine vs. human_).

We will target an 80% of effectiveness for our algorithm.

In [6]:
spam_sms_random = spam_sms.sample(frac=1, random_state=1)

training = spam_sms_random[:4458].reset_index(drop=True)
test = spam_sms_random[4458:].reset_index(drop=True)

print(training.shape)
print(test.shape)

(4458, 2)
(1114, 2)


In [7]:
training['Label'].value_counts(normalize=True)*100

ham     86.54105
spam    13.45895
Name: Label, dtype: float64

In [8]:
test['Label'].value_counts(normalize=True)*100

ham     86.804309
spam    13.195691
Name: Label, dtype: float64

We can see that the _ham_ and _spam_ percentages for both dataset replicate the behaviour in the original dataset, about 87% _ham_ and 13% _spam_, so we can consider to have a good randomized sample.

## Data cleaning

To calculate all the desired probabilities, we'll first need to perform a bit of data cleaning to bring the data in a format that will allow us to extract easily all the information we need. We will have to:

- Replace __SMS__ column with separate columns indicating unique words (vocabulary).
- Assign number of occurrences of each word in each message.
- Unify capitalization criteria.
- Remove punctuation characters.

In [9]:
training['SMS'] = training['SMS'].str.lower()
training['SMS'] = training['SMS'].str.replace('\W', ' ')
training.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


Now we can create a __vocabulary__ with unique words and generate the desired columns.

In [10]:
training['SMS'] = training['SMS'].str.split()

vocabulary = []

for sms in training['SMS']:
    for word in sms:
        vocabulary.append(word)

vocabulary = list(set(vocabulary))

In [11]:
word_counts_per_sms = {unique_word: [0] * len(training['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [12]:
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.shape

(4458, 7783)

In [13]:
training2 = pd.concat([training, word_counts], axis=1)
training2.head()

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


## Calculation

Now, with the dataset ready, we can start calculating probabilities. Let's begin with basics, __p\_spam__, __p\_ham__ and the lengths __N\_spam__, __N\_ham__ and __N\_vocabulary__. We will use Laplace smoothing, meaning alpha value will be 1.

In [14]:
training2_spam = training2[training2['Label'] == 'spam']
training2_ham = training2[training2['Label'] == 'ham']

p_spam = len(training2_spam) / len(training2)
p_ham = len(training2_ham) / len(training2)

n_spam = training2_spam['SMS'].apply(len)
n_spam = n_spam.sum()
n_ham = training2_ham['SMS'].apply(len)
n_ham = n_ham.sum()

n_vocabulary = len(vocabulary)

alpha = 1

## Parameters

We will calculate now the parameters needed for the algorithm in order to make it faster while filtering and analysing.

In [15]:
p_wi_given_spam = {}
p_wi_given_ham = {}

for word in vocabulary:
    p_wi_given_spam[word] = 0
    p_wi_given_ham[word] = 0

In [16]:
for word in vocabulary:
    n_word_given_spam = training2_spam[word].sum()
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha * n_vocabulary)
    p_wi_given_spam[word] = p_word_given_spam

    n_word_given_ham = training2_ham[word].sum()
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha * n_vocabulary)
    p_wi_given_ham[word] = p_word_given_ham

## Creating the spam filter

Now that we've calculated all the constants and parameters we need, we can start creating the spam filter. The spam filter can be understood as a function that:

- Takes in as input a new message (w1, w2, ..., wn)
- Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn)
- Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and classifies it accordingly.

In [17]:
def classify(message):
    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    for word in message:
        if word in p_wi_given_spam:
            p_spam_given_message *= p_wi_given_spam[word]
        if word in p_wi_given_ham:
            p_ham_given_message *= p_wi_given_ham[word]

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

Now we can test the function with two simple messages, that are obviously __spam__ and __ham__.

In [18]:
sms_spam = 'WINNER!! This is the secret code to unlock the money: C3421.'
sms_ham = 'Sounds good, Tom, then see u there'

In [19]:
classify(sms_spam)

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [20]:
classify(sms_ham)

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


On an initial instance, it seems to be working fine, but we will need to test it further.

## Measuring filter's accuracy

First off, we'll change the classify() function that we wrote previously to return the labels instead of printing them. Below, note that we now have return statements instead of print() functions.

In [21]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in p_wi_given_spam:
            p_spam_given_message *= p_wi_given_spam[word]

        if word in p_wi_given_ham:
            p_ham_given_message *= p_wi_given_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

Now we can use the function above to create a new column in the dataset with the machine's answer.

In [22]:
test['machine'] = test['SMS'].apply(classify_test_set)
test.head()

,Label,SMS,machine
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


Now we can compare the predicted values with the actual values to measure how good our spam filter is with classifying new messages. To make the measurement, we'll use __accuracy__ as a metric (correct over total).

In [23]:
correct = 0
total = len(test)

for row in test.iterrows():
    row = row[1]
    if row['Label'] == row['machine']:
        correct += 1
        
print('Accuracy:', 100*round(correct/total,6), '%')
print('Correct:', correct)
print('Incorrect:', total - correct)

Accuracy: 98.7433 %
Correct: 1100
Incorrect: 14


## Conclusions

The accuracy seems better than pretended initially, over 98%, when we were expecting at least 80%. So we managed to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm that resulted __very accurate__.